# import Library


In [54]:
import numpy as np
import pandas as pd

In [55]:
# IMPORTING PACKAGES

# FOR FILE
import zipfile

# DATA PREPROCESSING
import pandas as pd
import datetime 
import numpy as np
import re

# VISUALIZATION
import folium
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from matplotlib import pyplot as plt
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# MACHINE LEARNING PACKAGES
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.linear_model import Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [77]:
import geopandas as gpd
import movingpandas as mpd
import opendatasets as od
from os.path import exists
from shapely.geometry import Point

# Read data, and get preview of those datas

In [56]:
#loding data
file_Path_Sample ="C:/NewLaptopChenInfo/StonyBrookUniversityInfo/Courses/Fall2024/CSE487/Trajectory/Data_set/Trajctory/sampleSubmission.csv"
file_Path_Test ="C:/NewLaptopChenInfo/StonyBrookUniversityInfo/Courses/Fall2024/CSE487/Trajectory/Data_set/Trajctory/test.csv"
file_Path_Train = "C:/NewLaptopChenInfo/StonyBrookUniversityInfo/Courses/Fall2024/CSE487/Trajectory/Data_set/Trajctory/train.csv"
file_Path_GPSlocation = "C:/NewLaptopChenInfo/StonyBrookUniversityInfo/Courses/Fall2024/CSE487/Trajectory/Data_set/Trajctory/metaData_taxistandsID_name_GPSlocation.csv"

sample = pd.read_csv(file_Path_Sample)
train = pd.read_csv(file_Path_Train)
test = pd.read_csv(file_Path_Test)
location = pd.read_csv(file_Path_GPSlocation)

## 9 (nine) features, described as follows:

 ### 1. TRIP_ID(String) : 
   It contains an unique identifier for each trip;

 ### 2. CALL_TYPE (char):
   It identifies the way used to demand this service. It may contain one of three possible values:
      ‘A’ if this trip was dispatched from the central;
      ‘B’ if this trip was demanded directly to a taxi driver on a specific stand;
      ‘C’ otherwise (i.e. a trip demanded on a random street).

### 3. ORIGIN_CALL (integer):
   It contains an unique identifier for each phone number which was used to demand, at least, one service. It identifies the trip’s customer 
      if CALL_TYPE=’A’. Otherwise, it assumes a NULL value;

### 4. ORIGIN_STAND (integer): 
   It contains an unique identifier for the taxi stand. It identifies the starting point of the trip 
      if CALL_TYPE=’B’. Otherwise, it assumes a NULL value;

### 5. TAXI_ID (integer): 
   It contains an unique identifier for the taxi driver that performed each trip;
   
### 6. TIMESTAMP: (integer):
   Unix Timestamp (in seconds). It identifies the trip’s start; 

### 7. DAYTYPE: (char) 
   It identifies the daytype of the trip’s start. It assumes one of three possible values:
      ‘B’ if this trip started on a holiday or any other special day (i.e. extending holidays, floating holidays, etc.);
      ‘C’ if the trip started on a day before a type-B day;
      ‘A’ otherwise (i.e. a normal day, workday or weekend).

### 8. MISSING_DATA: (Boolean) 
   It is FALSE when the GPS data stream is complete and TRUE whenever one (or more) locations are missing.

### 9. POLYLINE: (String): 
   It contains a list of GPS coordinates (i.e. WGS84 format) mapped as a string. The beginning and the end of the string are identified with brackets (i.e. [ and ], respectively). Each pair of coordinates is also identified by the same brackets as [LONGITUDE, LATITUDE]. This list contains one pair of coordinates for [each 15 seconds] of trip. The last list item corresponds to the trip’s destination while the first one represents its start;

In [57]:
sample

,TRIP_ID,LATITUDE,LONGITUDE
0,T1,41.146504,-8.611317
1,T2,41.146504,-8.611317
2,T3,41.146504,-8.611317
3,T4,41.146504,-8.611317
4,T5,41.146504,-8.611317
...,...,...,...
315,T323,41.146504,-8.611317
316,T324,41.146504,-8.611317
317,T325,41.146504,-8.611317
318,T326,41.146504,-8.611317


In [58]:
train

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE
0,1372636858620000589,C,NaN,NaN,20000589,1372636858,A,False,"[[-8.618643,41.141412],[-8.618499,41.141376],[..."
1,1372637303620000596,B,NaN,7.0,20000596,1372637303,A,False,"[[-8.639847,41.159826],[-8.640351,41.159871],[..."
2,1372636951620000320,C,NaN,NaN,20000320,1372636951,A,False,"[[-8.612964,41.140359],[-8.613378,41.14035],[-..."
3,1372636854620000520,C,NaN,NaN,20000520,1372636854,A,False,"[[-8.574678,41.151951],[-8.574705,41.151942],[..."
4,1372637091620000337,C,NaN,NaN,20000337,1372637091,A,False,"[[-8.645994,41.18049],[-8.645949,41.180517],[-..."
...,...,...,...,...,...,...,...,...,...
1710665,1404171463620000698,C,NaN,NaN,20000698,1404171463,A,False,"[[-8.612469,41.14602],[-8.612487,41.145993],[-..."
1710666,1404171367620000670,C,NaN,NaN,20000670,1404171367,A,False,"[[-8.610138,41.140845],[-8.610174,41.140935],[..."
1710667,1388745716620000264,C,NaN,NaN,20000264,1388745716,A,False,[]
1710668,1404141826620000248,B,NaN,12.0,20000248,1404141826,A,False,"[[-8.630712,41.154885],[-8.63073,41.154813],[-..."


In [59]:
test

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE
0,T1,B,NaN,15.0,20000542,1408039037,A,False,"[[-8.585676,41.148522],[-8.585712,41.148639],[..."
1,T2,B,NaN,57.0,20000108,1408038611,A,False,"[[-8.610876,41.14557],[-8.610858,41.145579],[-..."
2,T3,B,NaN,15.0,20000370,1408038568,A,False,"[[-8.585739,41.148558],[-8.58573,41.148828],[-..."
3,T4,B,NaN,53.0,20000492,1408039090,A,False,"[[-8.613963,41.141169],[-8.614125,41.141124],[..."
4,T5,B,NaN,18.0,20000621,1408039177,A,False,"[[-8.619903,41.148036],[-8.619894,41.148036]]"
...,...,...,...,...,...,...,...,...,...
315,T323,A,70885.0,NaN,20000430,1419171485,A,False,"[[-8.570196,41.159484],[-8.570187,41.158962],[..."
316,T324,B,NaN,53.0,20000020,1419170802,A,False,"[[-8.613873,41.141232],[-8.613882,41.141241],[..."
317,T325,C,NaN,NaN,20000207,1419172121,A,False,"[[-8.6481,41.152536],[-8.647461,41.15241],[-8...."
318,T326,A,76232.0,NaN,20000667,1419171980,A,False,"[[-8.571699,41.156073],[-8.570583,41.155929],[..."


In [60]:
location

,ID,Descricao,Latitude,Longitude
0,1,Agra,41.1771457135,-8.609670
1,2,Alameda,41.15618964,-8.591064
2,3,Aldoar,41.1705249231,-8.665876
3,4,Alfândega,41.1437639911,-8.621803
4,5,Amial,41.1835097223,-8.612726
...,...,...,...,...
58,59,Tenente Valadim,41.1607030178,-8.641205
59,60,Trindade,41.1511237005,-8.609843
60,61,Vinte e Quatro de Agosto,41.1493101131,-8.600309
61,62,Viso,41.1778737602,-8.646558


In [61]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320 entries, 0 to 319
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   TRIP_ID       320 non-null    object 
 1   CALL_TYPE     320 non-null    object 
 2   ORIGIN_CALL   72 non-null     float64
 3   ORIGIN_STAND  123 non-null    float64
 4   TAXI_ID       320 non-null    int64  
 5   TIMESTAMP     320 non-null    int64  
 6   DAY_TYPE      320 non-null    object 
 7   MISSING_DATA  320 non-null    bool   
 8   POLYLINE      320 non-null    object 
dtypes: bool(1), float64(2), int64(2), object(4)
memory usage: 20.4+ KB


In [62]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1710670 entries, 0 to 1710669
Data columns (total 9 columns):
 #   Column        Dtype  
---  ------        -----  
 0   TRIP_ID       int64  
 1   CALL_TYPE     object 
 2   ORIGIN_CALL   float64
 3   ORIGIN_STAND  float64
 4   TAXI_ID       int64  
 5   TIMESTAMP     int64  
 6   DAY_TYPE      object 
 7   MISSING_DATA  bool   
 8   POLYLINE      object 
dtypes: bool(1), float64(2), int64(3), object(3)
memory usage: 106.0+ MB


# Missing info

## missing number

isnull(): chekc if the cells is Nan or not, if NaN return True; return a dataframe that is the same with our original data strcutre.

sum(): sum

Len(train): the total number of our trainig sample

（Total missing / Total rows）* 100 = pcent of missing


In [63]:
Pcent_missing_train = (train.isnull().sum()/len(train)) * 100
Pcent_missing_train

TRIP_ID          0.000000
CALL_TYPE        0.000000
ORIGIN_CALL     78.676776
ORIGIN_STAND    52.850111
TAXI_ID          0.000000
TIMESTAMP        0.000000
DAY_TYPE         0.000000
MISSING_DATA     0.000000
POLYLINE         0.000000
dtype: float64

In [64]:
Pcent_missing_test= (test.isnull().sum()/len(test)) * 100
Pcent_missing_test

TRIP_ID          0.0000
CALL_TYPE        0.0000
ORIGIN_CALL     77.5000
ORIGIN_STAND    61.5625
TAXI_ID          0.0000
TIMESTAMP        0.0000
DAY_TYPE         0.0000
MISSING_DATA     0.0000
POLYLINE         0.0000
dtype: float64

# other info pre processing

## Training data set

In [65]:
train["CALL_TYPE"].unique()

array(['C', 'B', 'A'], dtype=object)

In [66]:
train['DAY_TYPE'].unique()

array(['A'], dtype=object)

In [67]:
train['MISSING_DATA'].unique()

array([False,  True])

## Test Data set

In [68]:
test["CALL_TYPE"].unique()

array(['B', 'A', 'C'], dtype=object)

In [69]:
test['DAY_TYPE'].unique()

array(['A'], dtype=object)

In [70]:
test['MISSING_DATA'].unique()

array([False])

1. For both Test & Train, our DAY_TYPE is unique. Thus, it's useless for our code, remove it

2. For Training set, we have both True and False for our data. But in test only False(no missing GPS info)

## Drop Day_type for Both Dataset

In [71]:
train = train.drop("DAY_TYPE", axis=1)
test = test.drop("DAY_TYPE", axis=1)

In [72]:
train

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,MISSING_DATA,POLYLINE
0,1372636858620000589,C,NaN,NaN,20000589,1372636858,False,"[[-8.618643,41.141412],[-8.618499,41.141376],[..."
1,1372637303620000596,B,NaN,7.0,20000596,1372637303,False,"[[-8.639847,41.159826],[-8.640351,41.159871],[..."
2,1372636951620000320,C,NaN,NaN,20000320,1372636951,False,"[[-8.612964,41.140359],[-8.613378,41.14035],[-..."
3,1372636854620000520,C,NaN,NaN,20000520,1372636854,False,"[[-8.574678,41.151951],[-8.574705,41.151942],[..."
4,1372637091620000337,C,NaN,NaN,20000337,1372637091,False,"[[-8.645994,41.18049],[-8.645949,41.180517],[-..."
...,...,...,...,...,...,...,...,...
1710665,1404171463620000698,C,NaN,NaN,20000698,1404171463,False,"[[-8.612469,41.14602],[-8.612487,41.145993],[-..."
1710666,1404171367620000670,C,NaN,NaN,20000670,1404171367,False,"[[-8.610138,41.140845],[-8.610174,41.140935],[..."
1710667,1388745716620000264,C,NaN,NaN,20000264,1388745716,False,[]
1710668,1404141826620000248,B,NaN,12.0,20000248,1404141826,False,"[[-8.630712,41.154885],[-8.63073,41.154813],[-..."


In [73]:
test

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,MISSING_DATA,POLYLINE
0,T1,B,NaN,15.0,20000542,1408039037,False,"[[-8.585676,41.148522],[-8.585712,41.148639],[..."
1,T2,B,NaN,57.0,20000108,1408038611,False,"[[-8.610876,41.14557],[-8.610858,41.145579],[-..."
2,T3,B,NaN,15.0,20000370,1408038568,False,"[[-8.585739,41.148558],[-8.58573,41.148828],[-..."
3,T4,B,NaN,53.0,20000492,1408039090,False,"[[-8.613963,41.141169],[-8.614125,41.141124],[..."
4,T5,B,NaN,18.0,20000621,1408039177,False,"[[-8.619903,41.148036],[-8.619894,41.148036]]"
...,...,...,...,...,...,...,...,...
315,T323,A,70885.0,NaN,20000430,1419171485,False,"[[-8.570196,41.159484],[-8.570187,41.158962],[..."
316,T324,B,NaN,53.0,20000020,1419170802,False,"[[-8.613873,41.141232],[-8.613882,41.141241],[..."
317,T325,C,NaN,NaN,20000207,1419172121,False,"[[-8.6481,41.152536],[-8.647461,41.15241],[-8...."
318,T326,A,76232.0,NaN,20000667,1419171980,False,"[[-8.571699,41.156073],[-8.570583,41.155929],[..."


# Data Preparation

## Decode Time Signature Train Data
    beacuse for Time signature data, we are using unix timesteamp
    which are represent for how many second passed by after 1970/1/1 (UTC time)

    we need to decode that to human readable information by using Datetime package
    

Question: 
https://www.kaggle.com/code/hunter0007/trajectory-prediction-ensemble-aws-solution/comments

right here, he convert all cells into float time, but I didn't. However, the result are the same. so what's the point for converting?

In [74]:
train['converted_time']= [datetime.datetime.fromtimestamp(time,datetime.timezone.utc) for time in train["TIMESTAMP"]]

In [75]:
train['converted_time'].value_counts()

converted_time
2014-03-27 18:49:12+00:00    5
2014-03-27 19:31:22+00:00    5
2014-04-10 12:57:28+00:00    5
2013-10-17 13:02:53+00:00    4
2013-10-05 18:11:27+00:00    4
                            ..
2013-07-01 00:38:55+00:00    1
2013-07-01 00:11:22+00:00    1
2013-07-01 00:39:41+00:00    1
2013-07-01 00:22:41+00:00    1
2013-07-01 00:07:34+00:00    1
Name: count, Length: 1655366, dtype: int64

# PlayGround

## 1. draw the map:


In [82]:
train

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,MISSING_DATA,POLYLINE,converted_time
0,1372636858620000589,C,NaN,NaN,20000589,1372636858,False,"[[-8.618643,41.141412],[-8.618499,41.141376],[...",2013-07-01 00:00:58+00:00
1,1372637303620000596,B,NaN,7.0,20000596,1372637303,False,"[[-8.639847,41.159826],[-8.640351,41.159871],[...",2013-07-01 00:08:23+00:00
2,1372636951620000320,C,NaN,NaN,20000320,1372636951,False,"[[-8.612964,41.140359],[-8.613378,41.14035],[-...",2013-07-01 00:02:31+00:00
3,1372636854620000520,C,NaN,NaN,20000520,1372636854,False,"[[-8.574678,41.151951],[-8.574705,41.151942],[...",2013-07-01 00:00:54+00:00
4,1372637091620000337,C,NaN,NaN,20000337,1372637091,False,"[[-8.645994,41.18049],[-8.645949,41.180517],[-...",2013-07-01 00:04:51+00:00
...,...,...,...,...,...,...,...,...,...
1710665,1404171463620000698,C,NaN,NaN,20000698,1404171463,False,"[[-8.612469,41.14602],[-8.612487,41.145993],[-...",2014-06-30 23:37:43+00:00
1710666,1404171367620000670,C,NaN,NaN,20000670,1404171367,False,"[[-8.610138,41.140845],[-8.610174,41.140935],[...",2014-06-30 23:36:07+00:00
1710667,1388745716620000264,C,NaN,NaN,20000264,1388745716,False,[],2014-01-03 10:41:56+00:00
1710668,1404141826620000248,B,NaN,12.0,20000248,1404141826,False,"[[-8.630712,41.154885],[-8.63073,41.154813],[-...",2014-06-30 15:23:46+00:00


In [81]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1710670 entries, 0 to 1710669
Data columns (total 9 columns):
 #   Column          Dtype              
---  ------          -----              
 0   TRIP_ID         int64              
 1   CALL_TYPE       object             
 2   ORIGIN_CALL     float64            
 3   ORIGIN_STAND    float64            
 4   TAXI_ID         int64              
 5   TIMESTAMP       int64              
 6   MISSING_DATA    bool               
 7   POLYLINE        object             
 8   converted_time  datetime64[ns, UTC]
dtypes: bool(1), datetime64[ns, UTC](1), float64(2), int64(3), object(2)
memory usage: 106.0+ MB


In [89]:
df = pd.read_csv(file_Path_Train, nrows=30, usecols=['TRIP_ID', 'TAXI_ID', 'TIMESTAMP', 'MISSING_DATA', 'POLYLINE'])
df.POLYLINE = df.POLYLINE.apply(eval)  # string to list
df

,TRIP_ID,TAXI_ID,TIMESTAMP,MISSING_DATA,POLYLINE
0,1372636858620000589,20000589,1372636858,False,"[[-8.618643, 41.141412], [-8.618499, 41.141376..."
1,1372637303620000596,20000596,1372637303,False,"[[-8.639847, 41.159826], [-8.640351, 41.159871..."
2,1372636951620000320,20000320,1372636951,False,"[[-8.612964, 41.140359], [-8.613378, 41.14035]..."
3,1372636854620000520,20000520,1372636854,False,"[[-8.574678, 41.151951], [-8.574705, 41.151942..."
4,1372637091620000337,20000337,1372637091,False,"[[-8.645994, 41.18049], [-8.645949, 41.180517]..."
5,1372636965620000231,20000231,1372636965,False,"[[-8.615502, 41.140674], [-8.614854, 41.140926..."
6,1372637210620000456,20000456,1372637210,False,"[[-8.57952, 41.145948], [-8.580942, 41.145039]..."
7,1372637299620000011,20000011,1372637299,False,"[[-8.617563, 41.146182], [-8.617527, 41.145849..."
8,1372637274620000403,20000403,1372637274,False,"[[-8.611794, 41.140557], [-8.611785, 41.140575..."
9,1372637905620000320,20000320,1372637905,False,"[[-8.615907, 41.140557], [-8.614449, 41.141088..."


the reason we are using | df.POLYLINE = df.POLYLINE.apply(eval)

is beacuse our dataFrame for POLYLINE was like this:

data = {

    'POLYLINE': [
        "[[-8.610990, 41.14557], [-8.610977, 41.145579]]",
        "[[-8.611000, 41.145585], [-8.610987, 41.145594]]"
    ]
    
}

but we want to convert them into string like those:
(for example is our first ride info)

ride number-------------------POLYLINE

     1           [-8.610990, 41.14557], [-8.610977, 41.145579]
     
     1           [-8.611000, 41.145585], [-8.610987, 41.145594]

make a new df hold our POLYLINE

In [83]:
def unixtime_to_datetime(unix_time):
    return datetime.datetime.fromtimestamp(unix_time)
 
def compute_datetime(row):
    unix_time = row['TIMESTAMP']
    offset = row['running_number'] * datetime.timedelta(seconds=15)
    return unixtime_to_datetime(unix_time) + offset
 
def create_point(xy):
    try: 
        return Point(xy)
    except TypeError:  # when there are nan values in the input data
        return None

In [90]:
new_df = df.explode('POLYLINE')
new_df['geometry'] = new_df['POLYLINE'].apply(create_point)
new_df['running_number'] = new_df.groupby('TRIP_ID').cumcount()
new_df['datetime'] = new_df.apply(compute_datetime, axis=1)
new_df.drop(columns=['POLYLINE', 'TIMESTAMP', 'running_number'], inplace=True)
new_df

,TRIP_ID,TAXI_ID,MISSING_DATA,geometry,datetime
0,1372636858620000589,20000589,False,POINT (-8.618643 41.141412),2013-06-30 20:00:58
0,1372636858620000589,20000589,False,POINT (-8.618499 41.141376),2013-06-30 20:01:13
0,1372636858620000589,20000589,False,POINT (-8.620326 41.14251),2013-06-30 20:01:28
0,1372636858620000589,20000589,False,POINT (-8.622153 41.143815),2013-06-30 20:01:43
0,1372636858620000589,20000589,False,POINT (-8.623953 41.144373),2013-06-30 20:01:58
...,...,...,...,...,...
29,1372640555620000235,20000235,False,POINT (-8.620353 41.147352),2013-06-30 21:04:05
29,1372640555620000235,20000235,False,POINT (-8.620119 41.146893),2013-06-30 21:04:20
29,1372640555620000235,20000235,False,POINT (-8.620002 41.146812),2013-06-30 21:04:35
29,1372640555620000235,20000235,False,POINT (-8.619993 41.146839),2013-06-30 21:04:50


In [92]:
trajs = mpd.TrajectoryCollection(
    gpd.GeoDataFrame(new_df, crs=4326), 
    traj_id_col='TRIP_ID', obj_id_col='TAXI_ID', t='datetime')
trajs.hvplot(title='Kaggle Taxi Trajectory Data', tiles='CartoLight')

:Overlay
   .WMTS.I        :WMTS   [Longitude,Latitude]
   .Path.I        :Path   [Longitude,Latitude]
   .Path.II       :Path   [Longitude,Latitude]
   .Path.III      :Path   [Longitude,Latitude]
   .Path.IV       :Path   [Longitude,Latitude]
   .Path.V        :Path   [Longitude,Latitude]
   .Path.VI       :Path   [Longitude,Latitude]
   .Path.VII      :Path   [Longitude,Latitude]
   .Path.VIII     :Path   [Longitude,Latitude]
   .Path.IX       :Path   [Longitude,Latitude]
   .Path.X        :Path   [Longitude,Latitude]
   .Path.XI       :Path   [Longitude,Latitude]
   .Path.XII      :Path   [Longitude,Latitude]
   .Path.XIII     :Path   [Longitude,Latitude]
   .Path.XIV      :Path   [Longitude,Latitude]
   .Path.XV       :Path   [Longitude,Latitude]
   .Path.XVI      :Path   [Longitude,Latitude]
   .Path.XVII     :Path   [Longitude,Latitude]
   .Path.XVIII    :Path   [Longitude,Latitude]
   .Path.XIX      :Path   [Longitude,Latitude]
   .Path.XX       :Path   [Longitude,Latitude]
   .Path.XXI      :Path   [Longitude,Latitude]
   .Path.XXII     :Path   [Longitude,Latitude]
   .Path.XXIII    :Path   [Longitude,Latitude]
   .Path.XXIV     :Path   [Longitude,Latitude]
   .Path.XXV      :Path   [Longitude,Latitude]
   .Path.XXVI     :Path   [Longitude,Latitude]
   .Path.XXVII    :Path   [Longitude,Latitude]
   .Path.XXVIII   :Path   [Longitude,Latitude]
   .Path.XXIX     :Path   [Longitude,Latitude]
   .Path.XXX      :Path   [Longitude,Latitude]
   .Points.I      :Points   [Longitude,Latitude]   (triangle_angle)
   .Points.II     :Points   [Longitude,Latitude]   (triangle_angle)
   .Points.III    :Points   [Longitude,Latitude]   (triangle_angle)
   .Points.IV     :Points   [Longitude,Latitude]   (triangle_angle)
   .Points.V      :Points   [Longitude,Latitude]   (triangle_angle)
   .Points.VI     :Points   [Longitude,Latitude]   (triangle_angle)
   .Points.VII    :Points   [Longitude,Latitude]   (triangle_angle)
   .Points.VIII   :Points   [Longitude,Latitude]   (triangle_angle)
   .Points.IX     :Points   [Longitude,Latitude]   (triangle_angle)
   .Points.X      :Points   [Longitude,Latitude]   (triangle_angle)
   .Points.XI     :Points   [Longitude,Latitude]   (triangle_angle)
   .Points.XII    :Points   [Longitude,Latitude]   (triangle_angle)
   .Points.XIII   :Points   [Longitude,Latitude]   (triangle_angle)
   .Points.XIV    :Points   [Longitude,Latitude]   (triangle_angle)
   .Points.XV     :Points   [Longitude,Latitude]   (triangle_angle)
   .Points.XVI    :Points   [Longitude,Latitude]   (triangle_angle)
   .Points.XVII   :Points   [Longitude,Latitude]   (triangle_angle)
   .Points.XVIII  :Points   [Longitude,Latitude]   (triangle_angle)
   .Points.XIX    :Points   [Longitude,Latitude]   (triangle_angle)
   .Points.XX     :Points   [Longitude,Latitude]   (triangle_angle)
   .Points.XXI    :Points   [Longitude,Latitude]   (triangle_angle)
   .Points.XXII   :Points   [Longitude,Latitude]   (triangle_angle)
   .Points.XXIII  :Points   [Longitude,Latitude]   (triangle_angle)
   .Points.XXIV   :Points   [Longitude,Latitude]   (triangle_angle)
   .Points.XXV    :Points   [Longitude,Latitude]   (triangle_angle)
   .Points.XXVI   :Points   [Longitude,Latitude]   (triangle_angle)
   .Points.XXVII  :Points   [Longitude,Latitude]   (triangle_angle)
   .Points.XXVIII :Points   [Longitude,Latitude]   (triangle_angle)
   .Points.XXIX   :Points   [Longitude,Latitude]   (triangle_angle)
   .Points.XXX    :Points   [Longitude,Latitude]   (triangle_angle)